# Post-processing & Small mIoU functions

In [1]:
import numpy as np
from skimage import io, img_as_bool, measure, morphology
from skimage.draw import polygon, polygon2mask
import matplotlib.pyplot as plt
import os

import geopandas as gpd
import rasterio
import os

import pyproj
from shapely.geometry import box, Polygon
import shapely
from shapely.ops import transform, unary_union
import rasterio.features
import glob


## 1. Processing Small Building Predictions and GT Footprints: helper functions

### 1.1 Convert np.array to gpd

In [9]:
def get_footprint_gpd(mask, file_name, size):
    """
    Converts predictions from np.array to shapely polygons
    :mask: (np.array)
    :file_name: (str)
    :return: (gpd.GeoDataFrame)
    """
    
    tif_fp = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/raw_tif'
    with rasterio.open(os.path.join(tif_fp, f'{file_name}.tif')) as ds:
        t = ds.transform
        
    # Adjust for resolution
    if size == 512:
        factor = 1
    elif size == 1024:
        factor = 2
    else:
        raise NotImplemented('[ERROR] GPD Footprint --- Check resolution')
    
    shapes = rasterio.features.shapes(mask, connectivity=4)
    polygons = [shapely.geometry.Polygon(shape[0]["coordinates"][0]) for shape in shapes if shape[1] == 1]
    polygons = [shapely.affinity.affine_transform(geom, [t.a/factor,t.b,t.d,t.e/factor,t.xoff,t.yoff]) for geom in polygons]
    buildings = gpd.GeoDataFrame(geometry=polygons, crs='EPSG:26910')
    buildings = buildings.to_crs('EPSG:4326')
    return buildings

### 1.2 Pad buildings

In [10]:
# Pad small gt buildings
def pad_small_buildings(buildings_gpd, pad_buffer):
    buildings_gpd = buildings_gpd.to_crs(crs=3857)
    buildings_gpd.geometry = buildings_gpd.geometry.buffer(pad_buffer)
    buildings_gpd = buildings_gpd.to_crs('EPSG:4326')
    return buildings_gpd

### 1.3 Separate buildings

In [11]:
def separate_buildings(buildings_gpd, buffer_val):
    buildings_gpd = buildings_gpd.to_crs(crs=3857)
    buildings_gpd.geometry = buildings_gpd.geometry.buffer(-buffer_val)
    buildings_gpd = buildings_gpd.explode(index_parts=False)
    buildings_gpd.geometry = buildings_gpd.geometry.buffer(buffer_val)
    buildings_gpd = buildings_gpd.to_crs('EPSG:4326')
    return buildings_gpd

### 1.4 Filter roads

In [12]:
def filter_roads(buildings_gpd, file_name, road_buffer):
    tif_fp = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/raw_tif'
    oak_fp = '/oak/stanford/groups/deho/building_compliance/'
    zoning = gpd.read_file(os.path.join(oak_fp, 'san_jose_suppl', 'san_jose_Zoning_Districts.geojson'))
    zoning = zoning[(zoning['ZONINGABBREV'].str.contains('R-')) | \
                   ((zoning['ZONINGABBREV'] == 'A(PD)') & (zoning['PDUSE'] == 'Res'))]
    
    with rasterio.open(os.path.join(tif_fp, f'{file_name}.tif')) as inds:
        bounds = inds.bounds
        geom = box(*bounds)

    # prepare to convert TIF bounds to standard 4326
    wgs84 = pyproj.CRS('EPSG:26910') # LA is 11, SJ is 10
    utm = pyproj.CRS('EPSG:4326')

    project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform

    # convert
    utm_geom = shapely.ops.transform(project, geom)

    # clip the residential zones to just the TIF bounds
    zoning_clipped = gpd.clip(zoning, utm_geom)
    
    ## --- FILTER OUT PREDICTIONS ON ROADS (and other non-residential areas) ----
    zoning_clipped = zoning_clipped.to_crs(crs=3857)
    zoning_clipped = zoning_clipped.buffer(road_buffer)
    zoning_clipped = zoning_clipped.to_crs('EPSG:4326')
    
    buildings_gpd = gpd.clip(buildings_gpd, zoning_clipped)
    return buildings_gpd

### 1.5 Filter small buildings

In [13]:
def filter_buildings_area(buildings_gpd, area_thresh, larger_than=True):
    buildings_gpd = buildings_gpd.to_crs(crs=3857)
    buildings_gpd['area'] = buildings_gpd.area
    if larger_than:
        buildings_gpd = buildings_gpd.loc[buildings_gpd['area'] > area_thresh]
    else:
        buildings_gpd = buildings_gpd.loc[buildings_gpd['area'] < area_thresh]
    buildings_gpd = buildings_gpd.to_crs('EPSG:4326')
    return buildings_gpd

### 1.6 Convert gpd to np.array

In [14]:
def get_nparray_from_gpd(building_gpd, file_name, size):
    # Create empty array
    building_np = np.zeros((size, size), dtype="int64")

    tif_fp = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/raw_tif'
    with rasterio.open(os.path.join(tif_fp, f'{file_name}.tif')) as ds:
        t = ds.transform
    
    # Transform polygons to pixel space
    building_gpd = building_gpd.to_crs(crs='EPSG:26910')
    building_gpd = building_gpd.explode(column='geometry', index_parts=False, ignore_index=True)
    
    polygons = building_gpd.geometry
    #polygons = [shapely.affinity.affine_transform(
    #    geom, 
    #    [1/t.a, # a
    #     0, # b
    #     0, # d
    #     1/t.e, # e
    #     -t.xoff/t.a, #xOff
    #     -t.yoff/t.e]) for geom in polygons] #yOff
    
    # Adjust for resolution
    if size == 512:
        factor = 1
    elif size == 1024:
        factor = 2
    else:
        raise NotImplemented('[ERROR] GPD Footprint --- Check resolution')
    A = t.a / factor
    E = t.e / factor
    
    # Get inverse transformation
    k = 1/(1 - (t.b * t.d)/(A * E))
    polygons = [shapely.affinity.affine_transform(
    geom, 
    [k / A, # a
     -(t.b * k)/(A * E), # b
     -(t.d * k)/(E * A), # d
     1/E + (t.d * t.b * k)/(E * A * E), # e
     (k/A) * ((t.b *t.yoff)/(E) -t.xoff), #xOff
     -(t.d * k * t.b * t.yoff)/(E * A * E) + (t.d * k * t.xoff)/(E * A) - t.yoff/E]) for geom in polygons] #yOff
    
    # Create mask for each polygon
    for poly in polygons:
        a = poly.exterior.coords.xy
        poly_coords = np.array(list(zip(a[0], a[1])))
        poly_mask= polygon2mask((size, size), polygon=list(zip(a[1], a[0])))
        building_np += poly_mask
        
    # Flatten
    building_np = (building_np > 0).astype(np.int32)
    
    return building_np

### 1.6 Get (processed) small building predictions incorporating 1.1-1.5

In [15]:
# Get predicted small buildings
def get_pred_small_buildings(pred_image, file_name, buffer_val, small_area_thresh, 
                             large_area_thresh, road_buffer, np=True):    
    # Get gpd of building footprints
    inferred_buildings = get_footprint_gpd(mask=pred_image, file_name=file_name, size=pred_image.shape[0])
    
    # Separate closely connected buildings
    inferred_buildings = separate_buildings(buildings_gpd=inferred_buildings, buffer_val=buffer_val)
    
    # Filter out predictions on roads
    inferred_buildings = filter_roads(buildings_gpd=inferred_buildings, file_name=file_name, road_buffer=road_buffer)
    
    # Filter out predictions that are too small
    inferred_buildings = filter_buildings_area(
        buildings_gpd=inferred_buildings, area_thresh=small_area_thresh, larger_than=True)
    
    # Filter for small buildings
    inferred_buildings = filter_buildings_area(
        buildings_gpd=inferred_buildings, area_thresh=large_area_thresh, larger_than=False)
    
    if not np:
        return inferred_buildings
    
    # Convert from GPD to np.array
    pred_small_build = get_nparray_from_gpd(building_gpd=inferred_buildings, file_name=file_name, 
                                            size=pred_image.shape[0])
    
    return pred_small_build

## 2. Compute mIoU main function

In [18]:
def mIOU_manual(gt_image, pred_image):
    len_batch = len(gt_image)
    classes = np.unique(gt_image)

    mIOU = []



    for c in classes:
        curr_gt, curr_pred = (gt_image == c).astype(np.int32), (pred_image == c).astype(np.int32)

        overlap = np.sum(np.logical_and(curr_gt, curr_pred))
        n_gt = np.sum(curr_gt)
        n_pred = np.sum(curr_pred)

        iou = (overlap) / (n_gt + n_pred - overlap)

        mIOU.append( iou )

    mIOU = sum(mIOU) / len(classes)
    return mIOU

In [19]:
def small_mIOU(gt_image, pred_image, gt_area, file_name, pad_buffer, buffer_val, 
               small_area_thresh, large_area_thresh, road_buffer):
    len_batch = len(gt_image)
    classes = np.unique(gt_image)

    mIOU = []

    def compute_iou(gt, pred):
        overlap = np.sum(np.logical_and(gt, pred))
        n_gt = np.sum(gt)
        n_pred = np.sum(pred)
        return (overlap) / (n_gt + n_pred - overlap)

    # Get gt and predictions for each class
    bg_gt, bg_pred = (gt_image == 0).astype(np.int32), (pred_image == 0).astype(np.int32)
    build_gt, build_pred = (gt_image == 1).astype(np.int32), (pred_image == 1).astype(np.int32)

    # Class 0 (bg) ---------------------
    # * Get gt small buildings 
    #small_build_gt = ((gt_area < 115.) & (gt_area > 15)).astype(bool)
    build_gt_gpd = get_footprint_gpd(mask=gt_image, file_name=file_name, size=gt_image.shape[0])
    small_build_gt_gpd = filter_buildings_area(
        buildings_gpd=build_gt_gpd, area_thresh=small_area_thresh, larger_than=True)
    small_build_gt_gpd = filter_buildings_area(
        buildings_gpd=small_build_gt_gpd, area_thresh=large_area_thresh, larger_than=False)
    small_build_gt_np = get_nparray_from_gpd(small_build_gt_gpd, file_name, pred_image.shape[0])
    
    # * Pad gt small buildings
    padded_small_build_gt_gpd = pad_small_buildings(buildings_gpd=small_build_gt_gpd, pad_buffer=pad_buffer)
    padded_small_build_gt_np = get_nparray_from_gpd(padded_small_build_gt_gpd, file_name, pred_image.shape[0])

    # * Get predicted small buildings
    small_build_pred_np = get_pred_small_buildings(
        pred_image=pred_image, file_name=file_name, buffer_val=buffer_val, 
        small_area_thresh=small_area_thresh, large_area_thresh=large_area_thresh, road_buffer=road_buffer)
    
    # Invert
    bg_mask_np = ((padded_small_build_gt_np + small_build_pred_np) > 0).astype(np.int32)
    #bg_mask_np = 1 - bg_mask_np

    # * Mask predictions with padded gt small buildings and predicted small buildings
    masked_bg_pred = np.multiply(bg_pred, bg_mask_np)
    masked_bg_gt = np.multiply(bg_gt, bg_mask_np)

    bg_iou = compute_iou(masked_bg_gt, masked_bg_pred)
    mIOU.append(bg_iou)

    # Class 1 (building) ---------------------
    # * Mask predictions with small buildings
    small_build_pred_masked_np = np.multiply(build_pred, small_build_gt_np)

    # * Compute mIOU with adjusted gt/pred
    build_iou = compute_iou(small_build_gt_np, small_build_pred_masked_np)
    mIOU.append(build_iou)

    maps = {'class_0_gt': masked_bg_gt, 'class_0_pred': masked_bg_pred, 
            'class_1_gt': small_build_gt_np, 'class_1_pred': small_build_pred_masked_np, 
           'mask_bg': bg_mask_np, 'mask_bd': small_build_gt_np, 
           'mask_bg_pad': padded_small_build_gt_np, 'mask_bg_smallpred': small_build_pred_np,
           'trad_bg_gt': bg_gt, 'trad_bg_pred': bg_pred, 'trad_bd_gt': build_gt, 'trad_bd_pred': build_pred}
    return (sum(mIOU) / len(classes), maps)